# **Used Car Price Predictor Notebook**

### import required libraries

In [96]:
import keras 
import pandas as pd
import pandasql as ps
from geopy.geocoders import Nominatim
import time




## **MLOps Phase 1: Data Collection**
#### **Step 1: Extract Training Data from Parquet File**

In [118]:
df = pd.read_csv('used_cars_data.csv')

C:\Users\Daymo\AppData\Local\Temp\ipykernel_6104\1206154635.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('used_cars_data.csv')


#### **Step 2: Preview the Dataset's Schema**

In [131]:
pd.reset_option('all')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df.head(2)


,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,daysonmarket,dealer_zip,description,engine_cylinders,engine_displacement,engine_type,exterior_color,fleet,frame_damaged,franchise_dealer,franchise_make,front_legroom,fuel_tank_volume,fuel_type,has_accidents,height,highway_fuel_economy,horsepower,interior_color,isCab,is_certified,is_cpo,is_new,is_oemcpo,latitude,length,listed_date,listing_color,listing_id,longitude,main_picture_url,major_options,make_name,maximum_seating,mileage,model_name,owner_count,power,price,salvage,savings_amount,seller_rating,sp_id,sp_name,theft_title,torque,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,522,960,[!@@Additional Info@@!]Engine: 2.4L I4 ZERO EV...,I4,1300.0,I4,Solar Yellow,NaN,NaN,True,Jeep,41.2 in,12.7 gal,Gasoline,NaN,66.5 in,NaN,177.0,Black,NaN,NaN,NaN,True,NaN,18.3988,166.6 in,2019-04-06,YELLOW,237132766,-66.1582,https://static.cargurus.com/images/forsale/202...,['Quick Order Package'],Jeep,5 seats,7.0,Renegade,NaN,"177 hp @ 5,750 RPM",23141.0,NaN,0,2.8,370599.0,Flagship Chrysler,NaN,"200 lb-ft @ 1,750 RPM",A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,207,922,"[!@@Additional Info@@!]Keyless Entry,Ebony Mor...",I4,2000.0,I4,Narvik Black,NaN,NaN,True,Land Rover,39.1 in,17.7 gal,Gasoline,NaN,68 in,NaN,246.0,Black (Ebony),NaN,NaN,NaN,True,NaN,18.4439,181 in,2020-02-15,BLACK,265946296,-66.0785,https://static.cargurus.com/images/forsale/202...,['Adaptive Cruise Control'],Land Rover,7 seats,8.0,Discovery Sport,NaN,"246 hp @ 5,500 RPM",46500.0,NaN,0,3.0,389227.0,Land Rover San Juan,NaN,"269 lb-ft @ 1,400 RPM",A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


## **MLOPs Phase 2: Data Ingestion**
#### For the purposes of this tutorial, we will not be ingesting data. However, in an organizational setting, you'll use an ingestion framework, such as Apache Airflow, Apache Spark, or Apache Kafka, to collect data into a Data Lake to create a curated Training Dataset.

## **MLOPs Phase 3: Data Storage **
#### Since we already extracted this data was previously extracted from BigQuery, this phase has already been completed. Typically, you'll use a Cloud Storage Bucket (e.g., AWS S3, Google Cloud Storage) to store all data to be used in ML applications.

## **MLOPs Phase 4: Data Pipeline (Preparation and Wrangling)**
#### **Step 1: Use Intuition to Remove Irrelevant Features**

In [122]:
# columns_to_be_removed = ["vin","back_legroom","bed","bed_height","bed_length","body_type","cabin","city_fuel_economy","combine_fuel_economy","daysonmarket","description","engine_cylinders","engine_displacement","engine_type","franchise_dealer","franchise_make","front_legroom","fuel_tank_volume","fuel_type","height","highway_fuel_economy","horsepower","interior_color","isCab","is_certified","is_cpo","is_oemcpo","length","listed_date","listing_id","main_picture_url","maximum_seating","power","savings_amount","seller_rating","sp_id","sp_name","theft_title","torque","transmission","transmission_display","vehicle_damage_category","wheel_system_display","wheelbase","width"]

df_cleaned = df.drop(columns=columns_to_be_removed)
df_cleaned.tail(2)
# df_ = df_cleaned.tail(2)

,city,dealer_zip,exterior_color,fleet,frame_damaged,has_accidents,is_new,latitude,listing_color,longitude,major_options,make_name,mileage,model_name,owner_count,price,salvage,trimId,trim_name,wheel_system,year
3000038,Fairfield,94533,Green,False,False,False,False,38.2418,GREEN,-122.083,"['Sunroof/Moonroof', 'Navigation System', 'All...",Jaguar,27857.0,XE,1.0,26998.0,False,t65977,20d Premium AWD,AWD,2017
3000039,Napa,94559,Brilliant Silver,False,False,False,False,38.2927,SILVER,-122.278,"['Alloy Wheels', 'Bluetooth', 'Backup Camera',...",Nissan,22600.0,Rogue,1.0,19900.0,False,t76529,2017.5 SV FWD,FWD,2017


In [130]:
unique_zips = df_cleaned['dealer_zip'].unique()
numInt = 0
numStr = 0
non = 0
for i in unique_zips:
    if isinstance(i, int):
        numInt += 1
        

    elif isinstance(i, str):
        if len(i) == 1 or len(i) == 2:
            print(i)
        numStr += 1
    else:
        non += 1
print(f'numInt: {numInt}, numStr: {numStr}, non: {non}')


960
922
969
11706
10466
11797
7036
6108
11716
7643
11554
8701
11021
6110
7060
1119
11726
7644
6492
6830
11101
7608
1109
6902
10994
48843
48240
48442
48310
48219
6810
48328
48207
49203
10304
7470
11752
11743
10520
8812
11772
11570
48165
11023
6450
8691
7203
6479
10607
7604
48390
7727
48356
10710
12601
6460
7860
6705
6851
6066
48329
11758
10475
48015
7650
7503
10538
6704
49036
7522
11795
7105
48035
10019
7847
49017
6088
11413
11550
7111
7075
48423
11433
7901
11003
7606
11559
7801
11776
6051
8876
11787
7201
10801
11377
48134
10463
48340
11373
48317
11590
7803
11756
7304
11234
48066
7513
7001
1089
6002
60002
7403
10940
48104
11757
6708
48374
7022
43615
7410
7631
7424
91401
7663
10977
6405
10549
6040
11791
7072
7512
7026
49512
11801
48036
11704
6473
11355
6037
8690
11520
6111
48084
8822
48326
11793
7055
11581
6019
1720
8807
7202
7013
48044
48348
7416
48093
7095
11746
7456
10530
48336
6042
7446
6475
11780
11580
11803
44130
7067
7083
48220
7114
8904
7204
7205
7755
11236
48341
11763
48026
7652

In [114]:


# Initialize geocoder
geolocator = Nominatim(user_agent="geoapiExercises")

# Initialize a cache dictionary
geocode_cache = {}

# Function to get state from coordinates with caching
def get_state(row):
    # Check if the result is already in the cache
    coords = (row['latitude'], row['longitude'])
    if coords in geocode_cache:
        return geocode_cache[coords]
    
    location = geolocator.reverse(coords, exactly_one=True)
    address = location.raw.get('address', {})
    state = address.get('state', None)
    
    # Cache the result
    geocode_cache[coords] = state
    
    return state

# Apply the get_state function to each row in the DataFrame
df_['state'] = df_.apply(get_state, axis=1)

# Filter rows for USA and create "state" column
df_ = df_.dropna(subset=['state'])  # Remove rows where state couldn't be determined

# Filter and keep only USA rows
usa_states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
df_ = df_[df_['state'].isin(usa_states)]

# Print the updated DataFrame
print(df_)


GeocoderInsufficientPrivileges: Non-successful status code 403

In [92]:
df_cleaned.head(2)

,city,exterior_color,fleet,frame_damaged,has_accidents,is_new,latitude,listing_color,longitude,major_options,make_name,mileage,model_name,owner_count,price,salvage,trimId,trim_name,wheel_system,width,year
0,Bayamon,Solar Yellow,NaN,NaN,NaN,True,18.3988,YELLOW,-66.1582,['Quick Order Package'],Jeep,7.0,Renegade,NaN,23141.0,NaN,t83804,Latitude FWD,FWD,79.6 in,2019
1,San Juan,Narvik Black,NaN,NaN,NaN,True,18.4439,BLACK,-66.0785,['Adaptive Cruise Control'],Land Rover,8.0,Discovery Sport,NaN,46500.0,NaN,t86759,S AWD,AWD,85.6 in,2020


#### **Step 2: Remove Data with Troublesome Headers (Foreshadowing)**

#### **Step 4: Split "header" Column into "model_year", "make", and "model" Columns. Reorder Columns**

#### **Step 5: Clean "price" Column and Convert to Integer**

#### **Step 6: Clean "mileage" Column and Convert to Integer**

#### **Step 7: Split "colors" Column into "exterior_color" and "interior_color" Columns. Reorder Columns**

#### **Step 8: Split "condition" Column into "num_accidents", "num_owners", and "usage_type". Reorder Columns.**

#### **Step 9: Split "location" Column into "city" and "state" Columns. Reorder Columns.**

## **MLOps Phase 5: Exploratory Data Analysis (EDA)**

#### **Step 1: Create Histograms for Continuous, Numerical Features**

#### **Step 2: Create Correlation Matrix**

#### **Step 3: Show Descriptive Statistics for All Numeric Features**

#### **Step 4: Create Bar Charts for Categorical or Discrete Features**

## **MLOps Phase 6: Feature Engineering**

#### **Step 1: Remove All Outlier Values**

#### **Step 2: Preprocess Features**

#### **Step 3: Create DataFrame that Merges Pre-Preprocessed Data and Preprocessed Data**

## **MLOps Phase 7: Model Development**

#### **Step 1: Identify Features and Label**

#### **Step 2: Split Training Data into Training and Evaluation Subsets**

#### **Step 3: Fit Training Subset to Model**

#### **Step 4: Show Feature Importance Metrics**

#### **Step 5: Evaluate Model Performance**